In [9]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init


In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [16]:
print(device)

cpu


In [11]:
#parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [12]:
#MNIST dataset

mnist_train = dsets.MNIST(root='MNIST_data/',
                          train = True,
                          transform=transforms.ToTensor(),
                          download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform = transforms.ToTensor(),
                         download=True)

In [13]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True, 
                                          drop_last=True)

In [36]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
        nn.Conv2d(1,32,kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
        nn.Conv2d(32,64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(64,128,kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc1 = nn.Linear(3*3*128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x): # 통과해서 나온 x 가  self.layer2 통과
        out = self.layer1(x) 
        out = self.layer2(out)
        out = self.layer3(out)

        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [37]:
# model 만들기

model = CNN().to(device) # cuda가 사용 가능하면 cuda 아니면 cpu

print(model)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)


CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=1152, out_features=625, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=625, out_features=10, bias=True)
)


In [38]:
# training 

total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.to(device) # input data
        Y = Y.to(device) # label data 

        optimizer.zero_grad() # 매우 중요. 없으면 학습 안됨
        hypothesis = model(X) # 가설 <-모델에 인풋값 넣어서 

        cost = criterion(hypothesis, Y) # 가설과 라벨 사이의 차이를 계산 
        cost.backward()
        optimizer.step() # optimizer가 그 값을 토대로 계산 

        avg_cost += cost / total_batch

    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
print('Learning Finished!')

[Epoch:1] cost = 0.1658651977777481
[Epoch:2] cost = 0.04259182885289192
[Epoch:3] cost = 0.029343172907829285
[Epoch:4] cost = 0.022892946377396584
[Epoch:5] cost = 0.017668332904577255
[Epoch:6] cost = 0.013846314512193203
[Epoch:7] cost = 0.0130309434607625
[Epoch:8] cost = 0.010729193687438965
[Epoch:9] cost = 0.010261100716888905
[Epoch:10] cost = 0.008762439712882042
[Epoch:11] cost = 0.007738917600363493
[Epoch:12] cost = 0.007465306669473648
[Epoch:13] cost = 0.005962126888334751
[Epoch:14] cost = 0.007180104963481426
[Epoch:15] cost = 0.0058328621089458466
Learning Finished!


In [39]:
#test

with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9900000095367432
